In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)
import plotly.io as pio
pio.renderers.default = 'notebook'
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000
from sklearn import linear_model
import df_summary
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
df = pd.read_csv(r"C:\Users\gabri\Downloads\archive\Corona_NLP_train.csv",
                encoding="latin-1")

In [3]:
df.dropna(subset=['Location','TweetAt','OriginalTweet','Sentiment'], inplace=True)

In [4]:
df.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
5,3804,48756,"ÃT: 36.319708,-82.363649",16-03-2020,As news of the regionÂs first confirmed COVID...,Positive
6,3805,48757,"35.926541,-78.753267",16-03-2020,Cashier at grocery store was sharing his insig...,Positive


In [5]:
df['Sentiment_id'] = df['Sentiment'].factorize()[0]

In [6]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(df.OriginalTweet)

In [7]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
features = tfidf_transformer.fit_transform(X_train_counts).toarray()
features.shape

(32567, 68959)

In [8]:
# from sklearn.naive_bayes import MultinomialNB
# clf = MultinomialNB().fit(X_train_tfidf, df2.price_group)

In [9]:
# features = count_vect.get_feature_names()
# vocab = count_vect.vocabulary_
# clf.coef_
#tfidf_transformer.idf_

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier

#count_vect = CountVectorizer()
#X_train_counts = count_vect.fit_transform(df2['name'])

text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                          alpha=1e-3, random_state=42,
                          max_iter=5, tol=None)),
])

text_clf.fit(df.OriginalTweet, df.Sentiment_id)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                ('clf',
                 SGDClassifier(alpha=0.001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', loss='hinge',
                      

In [11]:
predicted = text_clf.predict(df.OriginalTweet)
np.mean(predicted == df.Sentiment_id)

0.7979549851076243

In [12]:
from sklearn.feature_selection import chi2

category_id_df = df[['Sentiment_id', 'Sentiment']].drop_duplicates().sort_values('Sentiment')
category_to_id = dict(category_id_df.values)

labels = df.Sentiment

N = 5
for category_id, Product in sorted(category_to_id.items()):
    features_chi2 = chi2(features, labels == Product)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(count_vect.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    print("# '{}':".format(Product))
    print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-N:])))
    print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-N:])))

# 'Neutral':
  . Most correlated unigrams:
. like
. demand
. hand
. panic
. no
  . Most correlated bigrams:
. 
# 'Positive':
  . Most correlated unigrams:
. scams
. like
. crisis
. hand
. panic
  . Most correlated bigrams:
. 
# 'Negative':
  . Most correlated unigrams:
. panic
. sanitizer
. no
. hand
. demand
  . Most correlated bigrams:
. 
# 'Extremely Positive':
  . Most correlated unigrams:
. hand
. safe
. thank
. best
. great
  . Most correlated bigrams:
. 
# 'Extremely Negative':
  . Most correlated unigrams:
. buying
. scams
. no
. panic
. crisis
  . Most correlated bigrams:
. 


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(df.Sentiment_id, predicted, target_names=labels.unique()))

In [ ]:
labels.unique()

In [68]:
df2[df2['name'].str.contains('bathroom', regex=False)]

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,price_group,price_group_predicted,price_group_name,room_type_id
208,54860,Great apartment with private bathroom and entr...,258164,Jenny,Manhattan,East Harlem,40.79958,-73.94275,Private room,100,5,204,2019-06-23,1.92,1,192,1,0,1,0
686,255024,"3BR, 2 bathroom condo in Bushwick",1340007,Roger,Brooklyn,Bushwick,40.70271,-73.91566,Entire home/apt,220,2,178,2019-06-24,2.69,1,297,2,0,2,1
921,342995,Unique private room and bathroom in Brownstone,1740216,Laura,Manhattan,Harlem,40.81169,-73.94355,Private room,75,2,77,2019-07-01,0.88,2,144,0,0,0,0
962,382523,A cozy Red Room with private bathroom,732535,William,Manhattan,Harlem,40.80491,-73.94866,Private room,160,3,10,2018-10-24,0.12,3,359,1,0,1,0
963,382524,Cozy room with private bathroom & outside garden,732535,William,Manhattan,Harlem,40.80334,-73.94805,Private room,180,3,14,2019-04-04,0.16,3,348,1,0,1,0
1174,501693,Room w/pvt bathroom on Central Park,1490833,Patrick,Manhattan,Upper West Side,40.79520,-73.96240,Private room,99,1,280,2019-06-23,3.51,1,246,0,0,0,0
1562,708374,private bedroom w/private bathroom on Central ...,315918,Arlette,Manhattan,Upper West Side,40.79854,-73.96089,Private room,100,3,326,2019-07-01,3.96,1,90,1,0,1,0
1678,762015,"Private Oasis, en suite bathroom",3992566,Clay,Brooklyn,Bushwick,40.70022,-73.93032,Private room,103,2,58,2019-06-20,0.94,3,69,1,0,1,0
1971,862890,Private bedroom w/ king size bed + pvt bathroom,4224309,Rachel...You Can Call Me Kaya :),Manhattan,Washington Heights,40.85081,-73.93118,Private room,150,2,3,2019-05-31,0.15,1,365,1,0,1,0
2000,882559,Private bedroom&bathroom in the heart of Manha...,4670155,Idan,Manhattan,Hell's Kitchen,40.76113,-73.98918,Private room,110,3,213,2019-06-24,2.79,1,281,1,0,1,0


In [11]:
# docs_new = df2['name']
# X_new_counts = count_vect.transform(docs_new)
# X_new_tfidf = tfidf_transformer.transform(X_new_counts)

# predicted = clf.predict(X_new_tfidf)

df2['price_group_predicted'] = predicted

C:\Users\gabri\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [12]:
df_comparison = df2[['name','price_group','price_group_predicted']]

In [13]:
df_comparison['correct'] = df_comparison['price_group'] == df_comparison['price_group_predicted']

C:\Users\gabri\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [14]:
def prepLinearModel(df, cols_remove = None):
    
    def isNumberic(dt):
        
        res = False
        
        numeric_list = [
            int,
            float,
            complex,
            np.int64,
            np.float64
        ]
        for num in numeric_list:
            if dt is num:
                return True
        return res
            
    #Drop from DataFrame passsed column list
    if cols_remove != None:
        try:
            prep_df = df.drop(cols_remove, axis=1)
        except:
            prep_df = df.copy()
            
    #Create dummy dimensions for categorical columns
    for col in prep_df.columns:
        if not isNumberic(prep_df[col].dtype.type):
            dims = prep_df[col].unique()
            if len(dims) > 1:
                for u_dim in dims[1:]:
                    prep_df[u_dim] = prep_df[col].apply(lambda x: x if (x == None) else (1 if (x==u_dim) else 0))
                prep_df.drop(col,axis=1,inplace=True)
    return prep_df

In [15]:
len(df['neighbourhood'].unique())

218

In [16]:
prep_df = prepLinearModel(df2, ['name','host_name','last_review'])

In [17]:
corr_matrix = prep_df.corr()['price']

In [18]:
corr_matrix.sort_values()

longitude                        -0.262822
Queens                           -0.148264
Bushwick                         -0.110700
Shared room                      -0.106296
Bedford-Stuyvesant               -0.094857
Bronx                            -0.079849
Washington Heights               -0.064288
Harlem                           -0.051854
Flatbush                         -0.047759
Flushing                         -0.047157
Ridgewood                        -0.045976
Astoria                          -0.045749
East Flatbush                    -0.045675
Crown Heights                    -0.044141
Sunnyside                        -0.042157
number_of_reviews                -0.041353
Staten Island                    -0.038602
Elmhurst                         -0.036872
Borough Park                     -0.035665
Inwood                           -0.034523
East Elmhurst                    -0.033929
Prospect-Lefferts Gardens        -0.033797
Jackson Heights                  -0.033368
Jamaica    

In [19]:
corr_df = pd.DataFrame(['']*len(corr_matrix), columns=['n'])
teh_mat = pd.concat([corr_df,corr_matrix], axis=1).dropna(subset=['price'])

In [20]:
def getVariabbles(m_df,predictee):
    free_variables = []
    
    y = m_df[predictee]
    for v in m_df.drop(predictee,axis=1).columns:
        x = m_df[v]
        free_variables.append(x)
        
        
    Y = np.vstack(y) #shape (N,) / (1d,)
    X = np.vstack(free_variables).T
    
    return Y,X

In [21]:
reg = linear_model.LinearRegression()
Y,X = getVariabbles(prep_df[[
                        'price',
                        'longitude',
                        'Queens',
                        'Shared room',
                        'price_group_predicted',
                        'Entire home/apt',
                        'Manhattan',
                        'calculated_host_listings_count',
                        'availability_365'
                    ]], 
                    'price'
                   )
#prep_df[['price',
#                            'Entire home/apt',
#                            'Manhattan',
#                             'Midtown',
#                             'Tribeca',
#                             'Queens',
#                             'calculated_host_listings_count',
#                            'availability_365',
#                             'Shared room',
#                            'longitude']]

In [22]:
reg.fit(X,Y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [23]:
reg.score(X,Y)

0.33264379068178884

In [24]:
reg.coef_

array([[-2.82689301e+02,  8.33583196e+00, -2.97810417e+01,
         2.40911138e+01,  8.41683851e+01,  3.24128976e+01,
        -1.42076148e-02,  1.12337702e-01]])